In [1]:
raw_time = ['../raw_data/서울_AWS_일일시간별_20250101_20250630.csv',
       '../raw_data/서울_AWS_일일시간별_20240101_20241231.csv',
       '../raw_data/서울_AWS_일일시간별_20230101_20231231.csv',
       '../raw_data/서울_AWS_일일시간별_20220101_20221231.csv']


raw_day = '../raw_data/서울_AWS_일일별_20220101_20250630.csv'

In [23]:
import pandas as pd
import numpy as np

# 2022~2025까지 년도별로 저장된 데이터
raw_time = [
    '../raw_data/서울_AWS_일일시간별_20250101_20250630.csv',
    '../raw_data/서울_AWS_일일시간별_20240101_20241231.csv',
    '../raw_data/서울_AWS_일일시간별_20230101_20231231.csv',
    '../raw_data/서울_AWS_일일시간별_20220101_20221231.csv'
]

# 년도별로 존재하는 파일들을 행 기준으로 병합 <- 그냥 각 파일을 25년도 데이터부터 밑으로 파일들을 합친것 aixs=0기준으로 합침(디폴트가 axis=0)
df_time = pd.concat([pd.read_csv(f, encoding='cp949') for f in raw_time], ignore_index=True)

# 날짜형 데이터로 변환 : 일시 데이터가 원래는 object타입으로 되어있음.
df_time['일시'] = pd.to_datetime(df_time['일시'], errors='coerce') # 일시: 문자열 형태의 시간 데이터를 datetime으로 변환
df_time['날짜'] = df_time['일시'].dt.date # 날짜: 시간 단위를 제거하고 날짜만 추출 (분석 기준 단위)




In [24]:
df_time

,지점,지점명,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),날짜
0,116,관악(레),2025-01-01 01:00:00,-3.9,260.4,1.0,0.0,71.0,2025-01-01
1,116,관악(레),2025-01-01 02:00:00,-3.4,257.2,1.0,0.0,80.0,2025-01-01
2,116,관악(레),2025-01-01 03:00:00,-2.7,243.2,1.4,0.0,81.0,2025-01-01
3,116,관악(레),2025-01-01 04:00:00,-2.0,232.2,1.9,0.0,82.0,2025-01-01
4,116,관악(레),2025-01-01 05:00:00,-1.6,233.2,1.7,0.0,82.0,2025-01-01
...,...,...,...,...,...,...,...,...,...
855392,889,현충원,2022-12-31 19:00:00,-3.0,207.4,0.6,0.0,90.0,2022-12-31
855393,889,현충원,2022-12-31 20:00:00,-3.2,228.8,0.7,0.0,93.0,2022-12-31
855394,889,현충원,2022-12-31 21:00:00,-3.9,237.4,0.9,0.0,95.0,2022-12-31
855395,889,현충원,2022-12-31 22:00:00,-3.3,225.5,0.7,0.0,97.0,2022-12-31


In [4]:
df1 = pd.read_csv(raw_time[0], encoding='cp949')
df2 = pd.read_csv(raw_time[1], encoding='cp949')
df3 = pd.read_csv(raw_time[2], encoding='cp949')
df4 = pd.read_csv(raw_time[3], encoding='cp949')

len(df1)+len(df2)+len(df3)+len(df4)

855397

In [5]:
df_time['지점명'].value_counts() # 1277일동안 빠지지 않고 측정을 했다면 1277*24(시간)=30648 이어야함 -> 누락된 값들이 존재한다는 뜻

지점명
기상청      30644
중구       30643
관악(레)    30640
강북*      30635
강동       30633
금천       30625
성동       30623
영등포      30623
중랑       30623
강남       30619
현충원      30611
성북       30608
송파       30604
동대문      30600
서초       30596
양천       30588
관악       30583
한강       30580
용산       30570
구로       30564
도봉       30548
서대문      30548
강서       30505
광진       30466
노원       30449
은평       30384
남현       30349
마포       29936
Name: count, dtype: int64

In [6]:
### 완전 누락일자: 하루 24시간 중 관측값이 하나도 없는 날 / 부분 누락일자: 관측값이 1개 이상 23개 이하인 날
'''
1. df_time에서 '일시'를 날짜로 변환
2. '지점명', '날짜' 단위로 관측횟수를 count()
3. 날짜별 24개 미만 → 부분누락, 0개 → 완전누락
4. df_time에 존재하지 않는 날짜(완전 누락)는 MultiIndex 기준틀을 만들어 fillna(0)으로 탐지
'''

# 전체 날짜 및 지점 조합 생성
all_dates = pd.date_range("2022-01-01", "2025-06-30").date
all_stations = df_time['지점명'].unique()
full_index = pd.MultiIndex.from_product([all_stations, all_dates], names=['지점명', '날짜'])

# 지점-날짜별 관측 횟수 집계
observed_counts = df_time.groupby(['지점명', '날짜'])['일시'].count()

# 누락 포함하여 병합 후 결측은 0으로
all_counts = pd.DataFrame(index=full_index).join(observed_counts).fillna(0)
all_counts = all_counts.rename(columns={'일시': '관측횟수'})

# 지점별 누락일수 계산
partial_missing = all_counts[(all_counts['관측횟수'] < 24) & (all_counts['관측횟수'] > 0)].groupby('지점명').size()
full_missing = all_counts[all_counts['관측횟수'] == 0].groupby('지점명').size()

compare = pd.DataFrame({
    '부분누락일수': partial_missing,
    '완전누락일수': full_missing
}).fillna(0).astype(int)

# 결과 요약 출력
print("지점별 누락 일수 요약:")
print(compare.sort_values(by='완전누락일수', ascending=False))

# 지점별 누락 날짜 리스트 생성
missing_info = {}

for station in all_stations:
    # 날짜별 관측 수 추출
    station_counts = all_counts.loc[station]['관측횟수']
    
    # 누락 날짜 분리
    완전누락 = station_counts[station_counts == 0].index.tolist()
    부분누락 = station_counts[(station_counts > 0) & (station_counts < 24)].index.tolist()
    
    # 저장
    missing_info[station] = {
        '완전누락일자': 완전누락,
        '부분누락일자': 부분누락
    }

print("지점별 누락 일자 리스트 생성 완료.")
missing_info

지점별 누락 일수 요약:
       부분누락일수  완전누락일수
지점명                  
마포         20      24
남현         15       8
은평         13       8
노원          6       7
광진          9       5
서대문         9       3
도봉         10       2
용산          8       2
구로         12       1
관악          9       1
양천         10       1
강서         20       1
금천          7       0
기상청         4       0
관악(레)       5       0
강북*         5       0
강동          8       0
강남         16       0
서초         10       0
동대문         9       0
송파          8       0
성북         10       0
성동          9       0
영등포         7       0
중구          5       0
중랑          9       0
한강         16       0
현충원        12       0
지점별 누락 일자 리스트 생성 완료.


{'관악(레)': {'완전누락일자': [],
  '부분누락일자': [datetime.date(2022, 1, 1),
   datetime.date(2023, 1, 1),
   datetime.date(2023, 10, 20),
   datetime.date(2024, 1, 1),
   datetime.date(2025, 1, 1)]},
 '강남': {'완전누락일자': [],
  '부분누락일자': [datetime.date(2022, 1, 1),
   datetime.date(2022, 9, 26),
   datetime.date(2022, 11, 24),
   datetime.date(2023, 1, 1),
   datetime.date(2023, 6, 1),
   datetime.date(2023, 6, 8),
   datetime.date(2023, 7, 24),
   datetime.date(2023, 11, 2),
   datetime.date(2024, 1, 1),
   datetime.date(2024, 4, 20),
   datetime.date(2024, 5, 9),
   datetime.date(2024, 5, 30),
   datetime.date(2024, 12, 23),
   datetime.date(2025, 1, 1),
   datetime.date(2025, 6, 9),
   datetime.date(2025, 6, 19)]},
 '서초': {'완전누락일자': [],
  '부분누락일자': [datetime.date(2022, 1, 1),
   datetime.date(2022, 2, 26),
   datetime.date(2022, 9, 23),
   datetime.date(2022, 12, 20),
   datetime.date(2023, 1, 1),
   datetime.date(2024, 1, 1),
   datetime.date(2024, 5, 5),
   datetime.date(2024, 11, 28),
   dateti

In [7]:
import json

# 날짜(Date) 타입을 문자열로 변환해서 저장
with open("../results/서울_AWS_지점별_누락일자_시간기준.json", "w", encoding="utf-8") as f:
    json.dump(missing_info, f, ensure_ascii=False, indent=2, default=str)

print("누락일자 정보가 JSON 파일로 저장되었습니다.")


누락일자 정보가 JSON 파일로 저장되었습니다.


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121496 entries, 0 to 121495
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   지점       121496 non-null  int64  
 1   지점명      121496 non-null  object 
 2   일시       121496 non-null  object 
 3   기온(°C)   121085 non-null  float64
 4   풍향(deg)  121390 non-null  float64
 5   풍속(m/s)  121392 non-null  float64
 6   강수량(mm)  120782 non-null  float64
 7   습도(%)    121183 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 7.4+ MB


In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244956 entries, 0 to 244955
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   지점       244956 non-null  int64  
 1   지점명      244956 non-null  object 
 2   일시       244956 non-null  object 
 3   기온(°C)   244477 non-null  float64
 4   풍향(deg)  243600 non-null  float64
 5   풍속(m/s)  243475 non-null  float64
 6   강수량(mm)  242310 non-null  float64
 7   습도(%)    244632 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 15.0+ MB


In [10]:
# 풍향은 평균이 무조건 좋은게 아니라 벡터 평균 (삼각함수 이용)이 일반적임
def average_wind_direction(wind_deg_series):
    # NaN 제거
    wind_deg_series = wind_deg_series.dropna()
    if wind_deg_series.empty:
        return np.nan
    
    # degree → radian 변환
    radians = np.deg2rad(wind_deg_series)
    
    # 벡터 성분 계산
    sin_sum = np.sum(np.sin(radians))
    cos_sum = np.sum(np.cos(radians))
    
    # 평균 방향 계산
    mean_rad = np.arctan2(sin_sum, cos_sum)
    mean_deg = np.rad2deg(mean_rad)
    
    # 0~360 범위 보정
    return mean_deg % 360

# 일자별 함수 (시각 제외, NaN 유지) skipna=True: 결측값을 제거하고 계산. 단, 전부 NaN이면 결과도 NaN
def daily_summary(group):
    return pd.Series({
        '평균기온(°C)': group['기온(°C)'].mean(skipna=True),
        '최저기온(°C)': group['기온(°C)'].min(skipna=True),
        '최고기온(°C)': group['기온(°C)'].max(skipna=True),
        '일강수량(mm)': group['강수량(mm)'].sum(skipna=True),
        '평균풍속(m/s)': group['풍속(m/s)'].mean(skipna=True),
        '벡터평균풍향(deg)': average_wind_direction(group['풍향(deg)']),
        '평균습도(%)': group['습도(%)'].mean(skipna=True),
        '최저습도(%)': group['습도(%)'].min(skipna=True),
        '최고습도(%)': group['습도(%)'].max(skipna=True),
        '습도표준편차': group['습도(%)'].std(skipna=True)
    })

# 일자별+지점별로 묶기
summary_time = df_time.groupby(['지점', '지점명', '날짜']).apply(daily_summary).reset_index()

summary_time.to_csv('../results/서울_AWS_시간을일자로_20220101_20250630.csv', index=False)

C:\Users\user\AppData\Local\Temp\ipykernel_16580\1118538960.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_time = df_time.groupby(['지점', '지점명', '날짜']).apply(daily_summary).reset_index()


In [11]:
# 일일자료 (최대 순간 풍속, 풍향 포함)
df_day = pd.read_csv('../raw_data/서울_AWS_일일별_20220101_20250630.csv', encoding='cp949')
df_day # 1277일*28개지점 = 35756 행 이 나와야하는데 (35686,10) -> 누락일자 확인 필요

,지점,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 풍속(m/s),최대 순간 풍속 풍향(deg)
0,116,관악(레),2022-01-01,-6.3,-12.3,-2.2,0.0,12.4,4.0,28.1
1,116,관악(레),2022-01-02,-4.1,-7.3,-1.5,0.0,10.9,3.7,241.9
2,116,관악(레),2022-01-03,-4.6,-8.5,-1.9,0.0,9.2,2.5,255.9
3,116,관악(레),2022-01-04,-6.2,-8.8,-2.7,0.0,12.4,3.5,19.7
4,116,관악(레),2022-01-05,-4.1,-7.4,-1.7,0.0,6.4,1.9,163.1
...,...,...,...,...,...,...,...,...,...,...
35681,889,현충원,2025-06-26,22.3,20.2,26.2,0.5,4.3,0.7,233.4
35682,889,현충원,2025-06-27,25.2,20.1,31.4,0.0,4.1,0.5,14.1
35683,889,현충원,2025-06-28,26.5,23.9,29.8,0.0,4.3,0.7,247.5
35684,889,현충원,2025-06-29,27.2,23.9,32.3,0.0,6.4,1.0,227.8


In [12]:
### 누락일자 확인
# 날짜
df_day['날짜'] = pd.to_datetime(df_day['일시'], errors='coerce').dt.date

# 전체 지점명 & 날짜
all_dates = pd.date_range("2022-01-01", "2025-06-30").date
all_stations = df_day['지점명'].unique()

# 기대되는 전체 조합
expected_index = pd.MultiIndex.from_product([all_stations, all_dates], names=['지점명', '날짜'])

# 실제 존재하는 조합
actual_index = pd.MultiIndex.from_frame(df_day[['지점명', '날짜']])

# 누락된 조합 확인
missing_index = expected_index.difference(actual_index)

# 누락된 조합을 데이터프레임으로 출력
missing_df = missing_index.to_frame(index=False)

print(f"누락된 지점-날짜 조합 개수: {len(missing_df)}")
missing_df


누락된 지점-날짜 조합 개수: 70


,지점명,날짜
0,강서,2023-10-21
1,강서,2023-10-22
2,관악,2024-09-22
3,광진,2024-05-03
4,광진,2024-05-04
...,...,...
65,은평,2023-12-08
66,은평,2023-12-09
67,은평,2023-12-10
68,은평,2023-12-11


In [13]:
missing_df.to_csv("../results/서울_AWS_지점별_누락일자_일자기준.csv", index=False, encoding="utf-8-sig")
print("누락 목록이 CSV 파일로 저장되었습니다.")


누락 목록이 CSV 파일로 저장되었습니다.


In [14]:
# 비교
overlap_summary = []

for station in missing_df['지점명'].unique():
    # 일일자료 기준 누락일자
    missing_daily = set(pd.to_datetime(missing_df[missing_df['지점명'] == station]['날짜']).dt.date)

    # 시간자료 기준 완전누락일자
    missing_hourly = set(pd.to_datetime(missing_info.get(station, {}).get('완전누락일자', [])).date)

    # 교집합 (겹치는 날짜)
    overlap = missing_daily & missing_hourly
    overlap_list = sorted(overlap)

    overlap_summary.append({
        "지점명": station,
        "일일자료누락": len(missing_daily),
        "시간자료_완전누락": len(missing_hourly),
        "겹치는날짜수": len(overlap),
        "겹치는날짜목록": ', '.join(str(d) for d in overlap_list)
    })

# 저장 및 출력
compare_df = pd.DataFrame(overlap_summary).sort_values(by="겹치는날짜수", ascending=False)
compare_df.to_csv("../results/서울_AWS_일일_vs_시간_누락비교_겹치는날짜포함.csv", index=False, encoding='utf-8-sig')

print("겹치는 날짜 목록 포함한 비교 결과 저장 완료")

compare_df


겹치는 날짜 목록 포함한 비교 결과 저장 완료


,지점명,일일자료누락,시간자료_완전누락,겹치는날짜수,겹치는날짜목록
8,마포,25,24,24,"2023-09-28, 2023-09-29, 2023-09-30, 2023-10-01..."
12,은평,8,8,8,"2023-12-05, 2023-12-06, 2023-12-07, 2023-12-08..."
4,남현,9,8,8,"2022-08-09, 2022-08-10, 2022-08-11, 2022-08-12..."
5,노원,7,7,7,"2023-10-27, 2023-10-28, 2023-10-29, 2023-10-30..."
2,광진,5,5,5,"2024-05-03, 2024-05-04, 2024-05-05, 2024-05-06..."
9,서대문,3,3,3,"2025-04-07, 2025-04-08, 2025-04-09"
11,용산,3,2,2,"2024-02-20, 2024-02-21"
6,도봉,2,2,2,"2023-12-03, 2024-05-12"
0,강서,2,1,1,2023-10-22
3,구로,3,1,1,2024-01-06


In [15]:
# 필요한 컬럼만 추출 + 컬럼명 변경
day_summary = df_day[[
    '지점', '지점명', '날짜',
    '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '일강수량(mm)',
    '최대 순간 풍속(m/s)', '최대 순간 풍속 풍향(deg)',
    '평균 풍속(m/s)'
]].rename(columns={
    '평균기온(°C)': '일_평균기온(°C)',
    '최저기온(°C)': '일_최저기온(°C)',
    '최고기온(°C)': '일_최고기온(°C)',
    '일강수량(mm)': '일_일강수량(mm)',
    '최대 순간 풍속(m/s)': '일_최대순간풍속(m/s)',
    '최대 순간 풍속 풍향(deg)': '일_최대순간풍속풍향(deg)',
    '평균 풍속(m/s)': '일_평균풍속(m/s)'
})

# 시간자료 + 일일자료 병합 : 누락값들이 있으니 outer join 사용
merged_df = pd.merge(
    summary_time,
    day_summary,
    on=['지점', '지점명', '날짜'],
    how='outer'
)

merged_df

,지점,지점명,날짜,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균풍속(m/s),벡터평균풍향(deg),평균습도(%),최저습도(%),최고습도(%),습도표준편차,일_평균기온(°C),일_최저기온(°C),일_최고기온(°C),일_일강수량(mm),일_최대순간풍속(m/s),일_최대순간풍속풍향(deg),일_평균풍속(m/s)
0,116,관악(레),2022-01-01,-7.052174,-12.3,-3.4,0.0,3.939130,9.387026,52.217391,32.0,84.0,14.437818,-6.3,-12.3,-2.2,0.0,12.4,28.1,4.0
1,116,관악(레),2022-01-02,-3.808333,-7.2,-2.0,0.0,3.891667,306.316084,71.541667,22.0,98.0,23.230376,-4.1,-7.3,-1.5,0.0,10.9,241.9,3.7
2,116,관악(레),2022-01-03,-4.962500,-8.1,-2.4,0.0,2.333333,275.261720,50.833333,15.0,98.0,31.080074,-4.6,-8.5,-1.9,0.0,9.2,255.9,2.5
3,116,관악(레),2022-01-04,-6.154167,-8.7,-2.7,0.0,3.529167,346.330197,62.833333,31.0,98.0,26.537368,-6.2,-8.8,-2.7,0.0,12.4,19.7,3.5
4,116,관악(레),2022-01-05,-4.525000,-7.2,-1.9,0.0,1.895833,124.928135,43.000000,35.0,60.0,6.494145,-4.1,-7.4,-1.7,0.0,6.4,163.1,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35688,889,현충원,2025-06-26,22.300000,20.2,25.4,0.5,0.716667,294.266456,81.958333,65.0,91.0,9.162680,22.3,20.2,26.2,0.5,4.3,233.4,0.7
35689,889,현충원,2025-06-27,24.845833,20.3,30.2,0.0,0.537500,4.904661,73.041667,52.0,90.0,13.845354,25.2,20.1,31.4,0.0,4.1,14.1,0.5
35690,889,현충원,2025-06-28,26.408333,24.0,29.1,0.0,0.762500,305.695393,79.333333,69.0,90.0,8.191547,26.5,23.9,29.8,0.0,4.3,247.5,0.7
35691,889,현충원,2025-06-29,27.479167,24.6,31.7,0.0,1.033333,231.654867,71.291667,52.0,88.0,10.609590,27.2,23.9,32.3,0.0,6.4,227.8,1.0


In [16]:
# 저장
merged_df.to_csv('../results/서울_AWS_일자_시간_병합_2022_2025.csv', index=False, encoding='utf-8-sig')

In [17]:
## 행정구 정보 추가 ##
data = pd.read_excel('../raw_data/서울_AWS_지점정보.xlsx', engine='openpyxl')
data

,AWS지점,지점명,주소,위도,경도,행정구,관리기관,좌표,운영여부
0,400,강남,서울특별시 강남구 일원동 580 탄천 물재생센터,37.49820,127.08162,강남구,기상청,WGS84,1
1,402,강동,서울특별시 강동구 고덕로 183 서울종합직업전문학교,37.55556,127.14498,강동구,기상청,WGS84,1
2,424,강북*,서울특별시 강북구 삼각산로 58 국립재활원 (수유동),37.63801,127.00981,강북구,기상청,WGS84,1
3,404,강서,서울특별시 강서구 양천로 201 서남물재생센터,37.57390,126.82953,강서구,기상청,WGS84,1
4,509,관악,서울특별시 관악구 관악로 1 서울대학교 대기환경관측소,37.45284,126.95015,관악구,기상청,WGS84,1
5,116,관악(레),경기도 과천시 중앙로 관악산길(관악산기상레이더관측소),37.44526,126.96402,과천시,기상청,WGS84,1
6,413,광진,서울특별시 광진구 자양2동 680-67,37.53344,127.08601,광진구,기상청,WGS84,1
7,423,구로,서울특별시 구로구 부일로 893 온수초등학교,37.49328,126.82629,구로구,기상청,WGS84,1
8,417,금천,서울특별시 금천구 시흥대로104길 31 독산초등학교,37.46551,126.90016,금천구,기상청,WGS84,1
9,410,기상청,서울특별시 동작구 여의대방로16길 61 기상청,37.49330,126.91746,동작구,기상청,WGS84,1


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AWS지점   30 non-null     int64  
 1   지점명     30 non-null     object 
 2   주소      29 non-null     object 
 3   위도      29 non-null     float64
 4   경도      29 non-null     float64
 5   행정구     29 non-null     object 
 6   관리기관    29 non-null     object 
 7   좌표      29 non-null     object 
 8   운영여부    30 non-null     int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.2+ KB


In [19]:
# 필요한 컬럼만 추출 (원하면)
data = data[['지점명', '주소', '위도', '경도', '행정구']]

# 병합: 지점명 기준 merged_df에 data를 left join
merged_with_meta = pd.merge(
    merged_df,
    data,
    on='지점명',
    how='left'
)

merged_with_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35693 entries, 0 to 35692
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   지점               35693 non-null  int64  
 1   지점명              35693 non-null  object 
 2   날짜               35693 non-null  object 
 3   평균기온(°C)         35654 non-null  float64
 4   최저기온(°C)         35654 non-null  float64
 5   최고기온(°C)         35654 non-null  float64
 6   일강수량(mm)         35693 non-null  float64
 7   평균풍속(m/s)        35621 non-null  float64
 8   벡터평균풍향(deg)      35624 non-null  float64
 9   평균습도(%)          33190 non-null  float64
 10  최저습도(%)          33190 non-null  float64
 11  최고습도(%)          33190 non-null  float64
 12  습도표준편차           33187 non-null  float64
 13  일_평균기온(°C)       35478 non-null  float64
 14  일_최저기온(°C)       35484 non-null  float64
 15  일_최고기온(°C)       35484 non-null  float64
 16  일_일강수량(mm)       35639 non-null  float64
 17  일_최대순간풍속(m/s

In [20]:
merged_with_meta

,지점,지점명,날짜,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균풍속(m/s),벡터평균풍향(deg),평균습도(%),...,일_최저기온(°C),일_최고기온(°C),일_일강수량(mm),일_최대순간풍속(m/s),일_최대순간풍속풍향(deg),일_평균풍속(m/s),주소,위도,경도,행정구
0,116,관악(레),2022-01-01,-7.052174,-12.3,-3.4,0.0,3.939130,9.387026,52.217391,...,-12.3,-2.2,0.0,12.4,28.1,4.0,경기도 과천시 중앙로 관악산길(관악산기상레이더관측소),37.44526,126.96402,과천시
1,116,관악(레),2022-01-02,-3.808333,-7.2,-2.0,0.0,3.891667,306.316084,71.541667,...,-7.3,-1.5,0.0,10.9,241.9,3.7,경기도 과천시 중앙로 관악산길(관악산기상레이더관측소),37.44526,126.96402,과천시
2,116,관악(레),2022-01-03,-4.962500,-8.1,-2.4,0.0,2.333333,275.261720,50.833333,...,-8.5,-1.9,0.0,9.2,255.9,2.5,경기도 과천시 중앙로 관악산길(관악산기상레이더관측소),37.44526,126.96402,과천시
3,116,관악(레),2022-01-04,-6.154167,-8.7,-2.7,0.0,3.529167,346.330197,62.833333,...,-8.8,-2.7,0.0,12.4,19.7,3.5,경기도 과천시 중앙로 관악산길(관악산기상레이더관측소),37.44526,126.96402,과천시
4,116,관악(레),2022-01-05,-4.525000,-7.2,-1.9,0.0,1.895833,124.928135,43.000000,...,-7.4,-1.7,0.0,6.4,163.1,1.9,경기도 과천시 중앙로 관악산길(관악산기상레이더관측소),37.44526,126.96402,과천시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35688,889,현충원,2025-06-26,22.300000,20.2,25.4,0.5,0.716667,294.266456,81.958333,...,20.2,26.2,0.5,4.3,233.4,0.7,서울특별시 동작구 현충로 210 국립서울현충원,37.50036,126.97652,동작구
35689,889,현충원,2025-06-27,24.845833,20.3,30.2,0.0,0.537500,4.904661,73.041667,...,20.1,31.4,0.0,4.1,14.1,0.5,서울특별시 동작구 현충로 210 국립서울현충원,37.50036,126.97652,동작구
35690,889,현충원,2025-06-28,26.408333,24.0,29.1,0.0,0.762500,305.695393,79.333333,...,23.9,29.8,0.0,4.3,247.5,0.7,서울특별시 동작구 현충로 210 국립서울현충원,37.50036,126.97652,동작구
35691,889,현충원,2025-06-29,27.479167,24.6,31.7,0.0,1.033333,231.654867,71.291667,...,23.9,32.3,0.0,6.4,227.8,1.0,서울특별시 동작구 현충로 210 국립서울현충원,37.50036,126.97652,동작구


In [21]:
# 저장
merged_with_meta.to_csv('../results/서울_AWS_일자_시간_지점정보포함_20220101_20250630.csv', index=False, encoding='utf-8-sig')